#  Hispasonic (data dumping; From csv to PostgreSQL)



<br>



Data migration stands as a pivotal step in the analytics process, and the transfer of information from a `CSV` file to a `PostgreSQL` database not only proves critical but marks the genesis of a transformative journey in the realm of data analytics. This process, an essential component of **ETL (Extraction, Transformation, Loading)**, doesn't just involve data transfer; it harnesses the advantages offered by `PostgreSQL`, a potent and versatile database.

<div style="text-align:center">
    <img src="images/csv_postgre.png" alt="2hand" style="width:350px;"/>
</div>

At the initial stage, data collection and cleansing within a `CSV` file lay the groundwork for analysis. While this file holds value, it operates within limitations in terms of management and processing. This is precisely where the data dump to `PostgreSQL` assumes a vital role. The robustness and scalability of `PostgreSQL` provide a conducive environment to store this preprocessed data, allowing for subsequent efficient and precise analysis.

The rationale behind this transition lies in `PostgreSQL's` ability to handle **substantial data volumes and support intricate queries**. Migrating to this relational **database enables deeper analysis, yields meaningful insights, and facilitates informed decision-making**.

This primary step, data dumping, is merely the commencement of a series of processes within the data analysis lifecycle. It's an integral part of the **ETL** phase, where information is extracted, transformed to suit analytical needs, and ultimately loaded into a robust repository like `PostgreSQL`, laying the foundation for comprehensive and accurate analysis.

In this article, we'll delve into the significance of this initial process, dissect the advantages of PostgreSQL as the destination for this data, and comprehend how this transition forms the fundamental pillar for unlocking the potential of information in the world of data analytics."

---

The aim to be achieved are:

- Establish communication with postgreSQL through jupyter notebooks.

- Dump the file `.csv` into the **PostgreSQL database**.




In [1]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine, func, MetaData, Table
import pymysql
import psycopg2

Next step is enable `load_ext sql`. This is not a standard command in Python, but a directive of the so-called `"IPython Magic Commands"`. 

These commands provide additional functionality within the IPython/Jupyter Notebook environment that are not part of the standard Python feature set. 

In this case, `'load_ext sql'` is used to load an extension that **allows SQL code to be executed directly inside a Jupyter Notebook cell**. This extension enables the execution of SQL queries in a Python kernel, allowing you to interact with a database directly from a code cell. 

To use `'load_ext sql'`, you must have the corresponding extension installed. Depending on the configuration of your Jupyter Notebook environment, you may need to install a specific extension that supports SQL execution within Python code cells.

In [2]:
%load_ext sql

### Ensure that the `db_user` has **super user** permissions to:

    $sudo -i -u postgres
    postgres@R7:~$ psql
    
    psql (15.5 (Ubuntu 15.5-1.pgdg22.04+1))
    Digite «help» para obtener ayuda.
    
    postgres=# \conninfo
    Está conectado a la base de datos «postgres» como el usuario «postgres» a través del socket en «/var/run/postgresql» port «5432».
    
    postgres=# \du
                                         Lista de roles
     Nombre de rol |                         Atributos                          | Miembro de 
    ---------------+------------------------------------------------------------+------------
     jupyter       | Crear BD                                                   | {}
     postgres      | Superusuario, Crear rol, Crear BD, Replicación, Ignora RLS | {}


### The `jupyter user` should have **super user** privileges for executing actions.

    postgres=# ALTER ROLE jupyter SUPERUSER;
    ALTER ROLE
    
    postgres=# \du
                                         Lista de roles
     Nombre de rol |                         Atributos                          | Miembro de 
    ---------------+------------------------------------------------------------+------------
     jupyter       | Superusuario, Crear BD                                     | {}
     postgres      | Superusuario, Crear rol, Crear BD, Replicación, Ignora RLS | {}
    
    postgres=# 
    
<br>

Because we're going to make use of the database from Jupyter using Python.

It is necessary to know that the connection would not be possible without the use of an external library. Fortunately, there are two available:


- `psycopg2`: It is a PostgreSQL database adapter for Python that provides a direct interface for working with PostgreSQL. It is highly efficient and is used directly to interact with the database, execute queries, handle transactions, etc. It is widely used due to its speed and functionality.


- `SQLAlchemy`: It is an object-relational mapping (ORM) library that provides a higher abstraction layer than psycopg2. It allows you to work with the database using Python objects and classes instead of directly writing SQL queries. SQLAlchemy is more flexible and offers different tools for interacting with the database.

This is how we can connect to the database using `psycopg2`

In [3]:
from os import listdir
from os.path import isfile, join

ruta = 'csv'
listado_csv = [f for f in listdir(ruta) if isfile(join(ruta, f))] # Comprueba los htmls
listado_csv

['hpw20231220_postgre.csv']

In [4]:
df = pd.read_csv("csv/hpw20231220_postgre.csv")
df.head(10)

,buy,change,sell,price,gift,search,repair,parts,synt_brand,description,city,published,expire,date_scrapped,seen
0,0,0,1,200,0,0,0,0,korg,korg 05r w módulo,Madrid,2023/07/27,2024/06/14,2023/12/20,456
1,0,0,1,360,0,0,0,0,waldorf,waldorf pulse rack,Barcelona,2023/12/18,2024/06/15,2023/12/20,88
2,0,0,1,495,0,0,0,0,roland,sampler s750 roland,Madrid,2023/09/30,2024/06/17,2023/12/20,440
3,0,1,0,0,0,0,0,0,roland,roland rd 2000,Madrid,2023/11/19,2024/06/12,2023/12/20,286
4,0,0,1,340,0,0,0,0,studiologic,studiologic numa compact 2,Castellón,2023/12/16,2024/06/13,2023/12/20,124
5,0,0,1,650,0,0,0,0,kurzweil,kurzweil pc3 le6,Madrid,2023/11/05,2024/06/17,2023/12/20,252
6,0,0,1,425,0,0,0,0,korg,korg drumlogue,Bizkaia,2023/01/24,2024/06/16,2023/12/20,1530
7,0,0,1,22,0,0,0,0,roland,sonidos roland jupiter x xm series y zenology ...,Pontevedra,2022/07/15,2024/06/16,2023/12/20,1744
8,0,0,1,90,0,0,0,0,doepfer,doepfer dual quantizer vintage edition,Madrid,2023/11/29,2024/06/13,2023/12/20,82
9,0,0,1,75,0,0,0,0,m-audio,controlador m-audio 5 octavas evolution mk461c,Madrid,2023/02/15,2024/06/16,2023/12/20,629


In [5]:
# Configure the connection to the database thru sqlalchemy

db_user = 'jupyter'
db_password = 'jjxx33pp'
db_name = 'postgres'
db_host = 'localhost'  # Change this if the database is on another host
db_port = '5432'       # Change this if the PostgreSQL port is different

# Create the connection string
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)
conn = engine.connect()

Comprobemos que la base de datos está vacia.

In [6]:
# Obtenemos la metadata y la tabla
metadata = MetaData(bind=engine)
hispasonic_table = Table('hispasonic', metadata, autoload=True, autoload_with=engine)

# Consulta para contar la cantidad de filas en la tabla
count_query = hispasonic_table.select().with_only_columns([func.count()])

# Ejecutar la consulta y obtener el resultado
count = engine.execute(count_query).scalar()

# Verificar si la tabla está vacía
if count == 0:
    print("The 'hispasonic' table is empty")
else:
    print(f"The 'hispasonic' table has {count} rows.")

The 'hispasonic' table has 1 rows.


In [7]:
conn = engine.connect()

# Export the dfFrame to the "hispasonic" table in the database
df.to_sql('hispasonic', engine, if_exists='append', index=False)

print("Data successfully exported to PostgreSQL.stgreSQL.")
# Cerrar la conexión
conn.close()

Data successfully exported to PostgreSQL.stgreSQL.


In [8]:
# Establecer una nueva conexión
conn = engine.connect()

# Realizar operaciones en la base de datos...
def consult(text):
    # Realiza la consulta
    consult = text
    return pd.read_sql(consult, engine)

#consult("DELETE FROM hispasonic;")

consult("SELECT * FROM hispasonic;")

,buy,change,sell,price,gift,search,repair,parts,synt_brand,description,city,published,expire,date_scrapped,seen,anon_user
0,0,0,1,200,0,0,0,0,korg,korg 05r w módulo,Madrid,2023/07/27,2024/06/14,2023/12/20,456,None
1,0,0,1,360,0,0,0,0,waldorf,waldorf pulse rack,Barcelona,2023/12/18,2024/06/15,2023/12/20,88,None
2,0,0,1,495,0,0,0,0,roland,sampler s750 roland,Madrid,2023/09/30,2024/06/17,2023/12/20,440,None
3,0,1,0,0,0,0,0,0,roland,roland rd 2000,Madrid,2023/11/19,2024/06/12,2023/12/20,286,None
4,0,0,1,340,0,0,0,0,studiologic,studiologic numa compact 2,Castellón,2023/12/16,2024/06/13,2023/12/20,124,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,0,1,600,0,0,0,0,korg,korg cx-3 mki analógico 1979 restaurado,Valencia,2023/10/06,2024/06/17,2023/12/20,657,None
268,0,0,1,1100,0,0,0,0,synthstrom,synthstrom deluge oled,Madrid,2023/12/20,2024/06/17,2023/12/20,90,None
269,0,0,1,90,0,0,0,0,behringer,behringer eurorack system-100 121 + 150,Barcelona,2023/12/06,2024/06/16,2023/12/20,158,None
270,0,0,1,190,0,0,0,0,korg,korg n5 fallo iluminación pantalla,Madrid,2023/03/31,2024/06/16,2023/12/20,702,None


In [9]:
conn.close()

We can see that the dataframe has been imported into the database correctly. Working with Jupyter's database isn't exactly convenient, so what you usually do is use software like:

"In the world of data analysis, there are several specialized tools for data visualization, and some of the most commonly used ones include:

1. **Tableau:** It's a popular tool for creating interactive visualizations and dashboards without the need for coding. It allows connecting multiple data sources and offers a wide range of visualization options.

2. **Power BI:** Developed by Microsoft, it's a powerful tool for creating interactive reports and dashboards. It enables data connection and visualization from various sources, along with analysis and modeling capabilities.

3. **QlikView and Qlik Sense:** These tools allow interactive data visualization and dashboard creation. They provide data discovery and associative analysis capabilities.

4. **Google Data Studio:** It's a free tool from Google that enables the creation of visually appealing reports and dashboards. It integrates well with other Google tools like Google Analytics and Google Sheets.

5. **Plotly:** Although it's a Python library, it also has an online graphical interface version that allows creating interactive visualizations directly on the web.

6. **D3.js:** It's a JavaScript library for creating advanced and customized visualizations. While it requires programming knowledge, it offers a high degree of flexibility and customization.

These tools enable the creation of interactive and appealing data visualizations, offering different levels of complexity and flexibility based on the data analyst's needs and the specific project requirements."